In [1]:
import os
import zarr
import random
import json
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch
import torchvision.transforms.functional as F
import random
import sys
from collections import defaultdict

warnings.filterwarnings("ignore")
sys.path.append("./src/")

from src.config import CFG
from src.dataloader import (
    read_zarr,
    read_info_json,
    scale_coordinates,
    create_dataset,
    create_segmentation_map,
    EziiDataset,
    drop_padding,
)
from src.network import UNet_2D, aug
from src.utils import save_images
from src.metric import score, create_cls_pos, create_cls_pos_sikii, create_df

sample_submission = pd.read_csv("../../inputs/sample_submission.csv")

In [2]:
train_dataset = EziiDataset(
    exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.train_zarr_types,
    train=True,
)

valid_dataset = EziiDataset(
    exp_names=CFG.valid_exp_names,
    # exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.valid_zarr_types,
    train=True,
)

test_dataset = EziiDataset(
    exp_names=["TS_6_4", "TS_5_4", "TS_69_2"],
    base_dir="../../inputs/test",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.valid_zarr_types,
    train=False,
)

from tqdm import tqdm

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


for row in tqdm(valid_loader):
    normalized_tomogram = row["normalized_tomogram"]
    break

[('TS_5_4', 'denoised'), ('TS_5_4', 'ctfdeconvolved'), ('TS_5_4', 'wbp'), ('TS_5_4', 'isonetcorrected'), ('TS_73_6', 'denoised'), ('TS_73_6', 'ctfdeconvolved'), ('TS_73_6', 'wbp'), ('TS_73_6', 'isonetcorrected'), ('TS_99_9', 'denoised'), ('TS_99_9', 'ctfdeconvolved'), ('TS_99_9', 'wbp'), ('TS_99_9', 'isonetcorrected'), ('TS_6_4', 'denoised'), ('TS_6_4', 'ctfdeconvolved'), ('TS_6_4', 'wbp'), ('TS_6_4', 'isonetcorrected'), ('TS_69_2', 'denoised'), ('TS_69_2', 'ctfdeconvolved'), ('TS_69_2', 'wbp'), ('TS_69_2', 'isonetcorrected')]
[('TS_86_3', 'denoised'), ('TS_6_6', 'denoised')]
[('TS_6_4', 'denoised'), ('TS_5_4', 'denoised'), ('TS_69_2', 'denoised')]


  0%|          | 0/2 [00:00<?, ?it/s]


In [3]:
class PadToSize(nn.Module):
    def __init__(self, resolution):
        super().__init__()
        if resolution == "0":
            self.size = 640
        elif resolution == "1":
            self.size = 320
        elif resolution == "2":
            self.size = 160

    def forward(self, x):
        return F.pad(x, (0, 0, self.size - x.shape[-1], self.size - x.shape[-2]))

In [4]:
model = UNet_2D().to("cuda")
model.eval()
model.load_state_dict(torch.load("./best_model.pth"))


optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(
    weight=torch.tensor([0.5, 32, 32, 32, 32, 32, 32]).to("cuda")
)
# criterion = DiceLoss()

best_model = None
best_loss = np.inf
batch_size = 4

valid_loss = []
valid_pred_tomogram = defaultdict(list)
valid_gt_tomogram = defaultdict(list)
model.eval()
tq = tqdm(range(len(valid_loader) * normalized_tomogram.shape[0]))
for data in valid_loader:
    exp_name = data["exp_name"][0]
    tomogram = data["normalized_tomogram"].to("cuda")
    segmentation_map = data["segmentation_map"].to("cuda").long()

    for i in range(tomogram.shape[1]):
        input_ = tomogram[:, i].unsqueeze(0)
        gt = segmentation_map[:, i]

        input_ = PadToSize(CFG.resolution)(input_)
        gt = PadToSize(CFG.resolution)(gt)
        output = model(input_)
        output = nn.functional.softmax(output, dim=1)
        loss = criterion(output, gt)

        valid_loss.append(loss.item())
        tq.set_description(f"Loss: {np.mean(valid_loss)}")
        tq.update(1)

        output = drop_padding(output, CFG.resolution)

        valid_pred_tomogram[exp_name].append(output.cpu().detach().numpy())
        valid_gt_tomogram[exp_name].append(gt.cpu().detach().numpy())
tq.close()

Loss: 1.4293923941643343: : 184it [00:04, 40.05it/s]                   


In [5]:
def create_gt_df(base_dir, exp_names):
    result_df = None
    particle_names = CFG.particles_name

    for exp_name in exp_names:
        for particle in particle_names:
            np_corrds = read_info_json(
                base_dir=base_dir, exp_name=exp_name, particle_name=particle
            )  # (n, 3)
            # 各行にexp_nameとparticle_name追加
            particle_df = pd.DataFrame(np_corrds, columns=["z", "y", "x"])
            particle_df["experiment"] = exp_name
            particle_df["particle_type"] = particle

            if result_df is None:
                result_df = particle_df
            else:
                result_df = pd.concat([result_df, particle_df], axis=0).reset_index(
                    drop=True
                )

    result_df = result_df.reset_index()  # index	experiment	particle_type	x	y	z
    result_df = result_df[["index", "experiment", "particle_type", "x", "y", "z"]]

    return result_df

In [6]:
gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", CFG.valid_exp_names)
gt_df = gt_df[gt_df["particle_type"] != "beta-amylase"].reset_index(drop=True)

# Validation

In [7]:
# pred_df = pd.read_csv("../../inputs/train_submission.csv")


def calc_score(initial_sikii):
    all_pred_df = None

    for exp_name in CFG.valid_exp_names:
        pred_tomogram = valid_pred_tomogram[exp_name]
        pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
        pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

        pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
            pred_tomogram, sikii_dict=initial_sikii
        )
        pred_df = create_df(pred_Ascale_pos, exp_name)
        # pred_df = create_df(pred_cls_pos, exp_name)

        if all_pred_df is None:
            all_pred_df = pred_df
        else:
            all_pred_df = pd.concat([all_pred_df, pred_df], axis=0).reset_index(
                drop=True
            )

    pred_df = all_pred_df[all_pred_df["particle_type"] != "beta-amylase"]
    pred_df = pred_df.drop_duplicates(subset=["x", "y", "z"], keep="first").reset_index(
        drop=True
    )

    pred_df = pred_df.reset_index()

    score_ = score(
        pred_df, gt_df, row_id_column_name="index", distance_multiplier=1, beta=4
    )

    return score_


def calc_score_by_exp(initial_sikii):
    exp_scores = {}

    for exp_name in CFG.valid_exp_names:
        gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", [exp_name])

        pred_tomogram = valid_pred_tomogram[exp_name]
        pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
        pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

        pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
            pred_tomogram, sikii_dict=initial_sikii
        )
        pred_df = create_df(pred_Ascale_pos, exp_name)

        pred_df = pred_df[pred_df["particle_type"] != "beta-amylase"]
        pred_df = pred_df.drop_duplicates(
            subset=["x", "y", "z"], keep="first"
        ).reset_index(drop=True)

        pred_df = pred_df.reset_index()

        score_ = score(
            pred_df, gt_df, row_id_column_name="index", distance_multiplier=1, beta=4
        )

        exp_scores[exp_name] = score_

    return exp_scores

In [8]:
constant = 0.35

initial_sikii = {
    "apo-ferritin": constant,
    "beta-amylase": constant,
    "beta-galactosidase": constant,
    "ribosome": constant,
    "thyroglobulin": constant,
    "virus-like-particle": constant,
}

score_ = calc_score(initial_sikii)
score_

0.3702277379631658

In [9]:
constant = 0.5666666666666667

initial_sikii = {
    "apo-ferritin": constant,
    "beta-amylase": constant,
    "beta-galactosidase": constant,
    "ribosome": constant,
    "thyroglobulin": constant,
    "virus-like-particle": constant,
}

score_ = calc_score_by_exp(initial_sikii)
score_

{'TS_86_3': 0.5155326189007919, 'TS_6_6': 0.5345474069874836}

In [10]:
best_sikii = 0
best_score = -np.inf

for sikii in np.linspace(0.3, 0.7, 100):
    initial_sikii = {
        "apo-ferritin": sikii,
        "beta-amylase": sikii,
        "beta-galactosidase": sikii,
        "ribosome": sikii,
        "thyroglobulin": sikii,
        "virus-like-particle": sikii,
    }
    score_ = calc_score(initial_sikii)
    if score_ > best_score:
        best_score = score_
        best_sikii = sikii
    print(sikii, score_)

best_sikii, best_score

NameError: name 'best_sikii' is not defined

# Train-Dataset

In [11]:
train_loss = []
valid_pred_tomogram = defaultdict(list)
valid_gt_tomogram = defaultdict(list)
model.eval()
tq = tqdm(range(len(train_loader) * normalized_tomogram.shape[0]))
for data in train_loader:
    exp_name = data["exp_name"][0]
    tomogram = data["normalized_tomogram"].to("cuda")
    segmentation_map = data["segmentation_map"].to("cuda").long()

    for i in range(tomogram.shape[1]):
        input_ = tomogram[:, i].unsqueeze(0)
        gt = segmentation_map[:, i]

        input_ = PadToSize(CFG.resolution)(input_)
        gt = PadToSize(CFG.resolution)(gt)
        output = model(input_)
        output = nn.functional.softmax(output, dim=1)

        tq.update(1)

        output = drop_padding(output, CFG.resolution)

        valid_pred_tomogram[exp_name].append(output.cpu().detach().numpy())
        valid_gt_tomogram[exp_name].append(gt.cpu().detach().numpy())
tq.close()

1840it [00:29, 63.00it/s]                      


In [12]:
def calc_score(initial_sikii):
    all_pred_df = None

    for exp_name in CFG.train_exp_names:
        pred_tomogram = valid_pred_tomogram[exp_name]
        pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
        pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

        pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
            pred_tomogram, sikii_dict=initial_sikii
        )
        pred_df = create_df(pred_Ascale_pos, exp_name)
        # pred_df = create_df(pred_cls_pos, exp_name)

        if all_pred_df is None:
            all_pred_df = pred_df
        else:
            all_pred_df = pd.concat([all_pred_df, pred_df], axis=0).reset_index(
                drop=True
            )

    pred_df = all_pred_df[all_pred_df["particle_type"] != "beta-amylase"]
    pred_df = pred_df.drop_duplicates(subset=["x", "y", "z"], keep="first").reset_index(
        drop=True
    )

    pred_df = pred_df.reset_index()

    score_ = score(
        pred_df, gt_df, row_id_column_name="index", distance_multiplier=1, beta=4
    )

    return score_

In [13]:
gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", CFG.train_exp_names)
gt_df = gt_df[gt_df["particle_type"] != "beta-amylase"].reset_index(drop=True)

In [ ]:
best_sikii = 0
best_score = -np.inf

for sikii in np.linspace(0.4, 0.7, 25):
    initial_sikii = {
        "apo-ferritin": sikii,
        "beta-amylase": sikii,
        "beta-galactosidase": sikii,
        "ribosome": sikii,
        "thyroglobulin": sikii,
        "virus-like-particle": sikii,
    }
    score_ = calc_score(initial_sikii)
    if score_ > best_score:
        best_score = score_
        best_sikii = sikii
    print(sikii, score_)

0.4 0.3534642243245894
0.41250000000000003 0.3633357267104117
0.42500000000000004 0.3683684864295206
0.4375 0.37240028369233763
0.45 0.378698423683563
0.4625 0.3855536346532354
0.475 0.3851317411342358
0.4875 0.38739060577678897
0.5 0.3940074160254892
0.5125 0.39966732042505076
0.525 0.4059922209302399
0.5375 0.41111633642684453
0.55 0.4149057778718746
0.5625 0.4166629709990723
0.575 0.4200151894119474
0.5874999999999999 0.41221797587577946
0.6 0.4219170897705994
0.6125 0.42937237746217577
0.625 0.43422966984334505
0.6375 0.4390913151810357
0.6499999999999999 0.4434392255622401
0.6625 0.4388315231214689
0.6749999999999999 0.4423836595504814
0.6875 0.45234293202416415


In [ ]:
best_sikii, best_score

## Test-Dataset

In [ ]:
valid_pred_tomogram = defaultdict(list)
model.eval()
tq = tqdm(range(len(test_loader) * normalized_tomogram.shape[0]))
for data in test_loader:
    exp_name = data["exp_name"][0]
    tomogram = data["normalized_tomogram"].to("cuda")

    for i in range(tomogram.shape[1]):
        input_ = tomogram[:, i].unsqueeze(0)

        input_ = PadToSize(CFG.resolution)(input_)
        output = model(input_)
        output = nn.functional.softmax(output, dim=1)

        tq.update(1)

        output = drop_padding(output, CFG.resolution)

        valid_pred_tomogram[exp_name].append(output.cpu().detach().numpy())
tq.close()

all_pred_df = None

for exp_name in ["TS_6_4", "TS_5_4", "TS_69_2"]:
    pred_tomogram = valid_pred_tomogram[exp_name]
    pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
    pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

    pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
        pred_tomogram, sikii_dict=initial_sikii
    )
    pred_df = create_df(pred_Ascale_pos, exp_name)
    # pred_df = create_df(pred_cls_pos, exp_name)

    if all_pred_df is None:
        all_pred_df = pred_df
    else:
        all_pred_df = pd.concat([all_pred_df, pred_df], axis=0).reset_index(drop=True)

pred_df = all_pred_df[all_pred_df["particle_type"] != "beta-amylase"]
pred_df = pred_df.drop_duplicates(subset=["x", "y", "z"], keep="first").reset_index(
    drop=True
)

pred_df = pred_df.reset_index()

In [ ]:
pred_df